# Astana Mobility GPS Dataset (No-Timestamp) Exploratory Analysis & Mapping

This notebook performs privacy-preserving exploratory geospatial analysis for a GPS mobility dataset (Astana, Kazakhstan) **without timestamps**.
Limitations: No temporal ordering beyond original file row sequence; can't compute durations, dwell times, speeds-over-ground validation, or ETA-like metrics.
Focus: hotspots, start/end clusters, OD corridors, sample routes, anomaly cues.
All outputs aggregated with k-anonymity (k>=20). Raw trajectories only shown for a masked 5-trip sample.

In [3]:
# -------------------------------------------------------------
# 0. (Optional) Environment Setup - Run if libraries missing
# -------------------------------------------------------------
# Adjust as needed; avoids network calls if already installed.
# NOTE: Some environments disallow installs inside notebooks.
# You can skip this cell if packages are present.
# Python 3.11 targeted.
install_cmd = False  # set True to attempt pip install
required_packages = [
    'pandas','numpy','matplotlib','geopandas','shapely','h3','pydeck','folium',
    'scikit-learn','geopy'
]
optional_packages = ['datashader','similaritymeasures','traj-dist','haversine']
if install_cmd:
    import sys, subprocess
    for pkg in required_packages + optional_packages:
        try:
            __import__(pkg.split('-')[0].replace('-', '_'))
        except Exception:
            print(f'Installing {pkg} ...')
            subprocess.run([sys.executable,'-m','pip','install',pkg])
else:
    print('Install step skipped. Set install_cmd=True to attempt installs.')

Install step skipped. Set install_cmd=True to attempt installs.


In [4]:
# -------------------------------------------------------------
# 1. Parameters & Configuration (re-added)
# -------------------------------------------------------------
import os, json, random, hashlib, warnings
warnings.filterwarnings('ignore')

DATA_PATH = 'geo_locations_astana_hackathon.csv'
OUTPUT_DIR = 'outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)

BBOX = {
    'min_lat': 50.5, 'max_lat': 51.8,
    'min_lng': 70.5, 'max_lng': 72.0
}

H3_RES_LIST = [7,8,9]
PRIMARY_HEAT_RES = 8
K_ANON = 20
SAMPLE_N_TRIPS = 5
TOP_K_OD = 5
ROUTE_SAMPLE_PER_OD = 15
DP_TOLERANCE_DEG = 0.0005  # ~55m simplification
# New: 20m smoothing tolerance in degrees (approx): 20m / 111320 ≈ 0.00018
DP_SMOOTH_TOL_DEG = 20.0 / 111320.0
STOP_SPEED_THRESHOLD = None
STOP_CLUSTER_EPS_M = 60  # original, may be overridden by new stop logic
STOP_CLUSTER_MIN_SAMPLES = 10  # original, replaced later
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
ROUGHNESS_Z_THRESHOLD = 2.5
HEADING_VAR_Z_THRESHOLD = 2.5

# Build CONFIG with only JSON-serializable values; stringify the rest defensively
from collections.abc import Mapping

def _to_jsonable(x):
    try:
        json.dumps(x)
        return x
    except TypeError:
        return str(x)

CONFIG = {k: _to_jsonable(v) for k,v in globals().items() if k.isupper()}
with open(os.path.join(OUTPUT_DIR,'config_snapshot.json'),'w') as f:
    json.dump(CONFIG, f, indent=2)
print('Configuration saved to outputs/config_snapshot.json (updated with DP_SMOOTH_TOL_DEG)')

Configuration saved to outputs/config_snapshot.json (updated with DP_SMOOTH_TOL_DEG)


In [5]:
# -------------------------------------------------------------
# 2. Imports & Optional Libraries + H3 Wrappers
# -------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from shapely.geometry import Point, LineString
import geopandas as gpd
try:
    import h3
except ImportError as e:
    raise ImportError('h3 library required. Install via pip install h3.') from e

# --- H3 API compatibility and wrapper functions ---
if not hasattr(h3, 'geo_to_h3') and hasattr(h3, 'latlng_to_cell'):
    h3.geo_to_h3 = h3.latlng_to_cell  # type: ignore
if not hasattr(h3, 'h3_to_geo') and hasattr(h3, 'cell_to_latlng'):
    h3.h3_to_geo = h3.cell_to_latlng  # type: ignore
if not hasattr(h3, 'h3_to_geo_boundary') and hasattr(h3, 'cell_to_boundary'):
    def _boundary_adapter(cell, geo_json=True):
        return h3.cell_to_boundary(cell, geo_json=geo_json)
    h3.h3_to_geo_boundary = _boundary_adapter  # type: ignore

def h3_cell_index(lat, lng, res):
    if hasattr(h3, 'geo_to_h3'):
        return h3.geo_to_h3(lat, lng, res)
    if hasattr(h3, 'latlng_to_cell'):
        return h3.latlng_to_cell(lat, lng, res)
    raise AttributeError('No valid H3 indexing function found.')

def h3_cell_center(cell):
    if hasattr(h3, 'h3_to_geo'):
        return h3.h3_to_geo(cell)
    if hasattr(h3, 'cell_to_latlng'):
        return h3.cell_to_latlng(cell)
    raise AttributeError('No valid H3 center function found.')

def h3_cell_boundary(cell):
    if hasattr(h3, 'h3_to_geo_boundary'):
        return h3.h3_to_geo_boundary(cell, geo_json=True)
    if hasattr(h3, 'cell_to_boundary'):
        return h3.cell_to_boundary(cell, geo_json=True)
    raise AttributeError('No valid H3 boundary function found.')

try:
    import pydeck as pdk
    HAS_PYDECK = True
except Exception:
    HAS_PYDECK = False

try:
    import folium
    HAS_FOLIUM = True
except Exception:
    HAS_FOLIUM = False

try:
    import datashader as ds, datashader.transfer_functions as tf
    from datashader.utils import lnglat_to_meters
    HAS_DATASHADER = True
except Exception:
    HAS_DATASHADER = False

# Distance utilities
try:
    from geopy.distance import geodesic
except Exception:
    geodesic = None

try:
    import similaritymeasures
    HAS_SIMILARITY = True
except Exception:
    HAS_SIMILARITY = False

try:
    from sklearn.cluster import DBSCAN
except Exception as e:
    raise ImportError('scikit-learn required for DBSCAN. Install scikit-learn.') from e

print(f'pydeck: {HAS_PYDECK}, folium: {HAS_FOLIUM}, datashader: {HAS_DATASHADER}, similarity: {HAS_SIMILARITY}')

pydeck: True, folium: True, datashader: True, similarity: True


In [6]:
# -------------------------------------------------------------
# 3. Helper Functions
# -------------------------------------------------------------
EARTH_RADIUS_M = 6371008.8

# Confirm H3 compatibility functions exist (defensive re-check)
if not hasattr(h3, 'geo_to_h3') and hasattr(h3, 'latlng_to_cell'):
    h3.geo_to_h3 = h3.latlng_to_cell  # type: ignore
if not hasattr(h3, 'h3_to_geo') and hasattr(h3, 'cell_to_latlng'):
    h3.h3_to_geo = h3.cell_to_latlng  # type: ignore
if not hasattr(h3, 'h3_to_geo_boundary') and hasattr(h3, 'cell_to_boundary'):
    def _boundary_adapter(cell, geo_json=True):
        return h3.cell_to_boundary(cell, geo_json=geo_json)
    h3.h3_to_geo_boundary = _boundary_adapter  # type: ignore

def normalize_bearing(b):
    if pd.isna(b): return np.nan
    b = b % 360.0
    if b < 0: b += 360.0
    return b

def haversine_km(lat1, lon1, lat2, lon2):
    # vectorized friendly (expects scalars here)
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return (EARTH_RADIUS_M * c)/1000.0

def line_length_km(coords):
    if len(coords) < 2: return 0.0
    return sum(haversine_km(coords[i][1], coords[i][0], coords[i+1][1], coords[i+1][0]) for i in range(len(coords)-1))

def simplify_linestring(coords, tolerance=DP_TOLERANCE_DEG):
    if len(coords) < 3: return coords
    ls = LineString(coords)
    simp = ls.simplify(tolerance, preserve_topology=False)
    return list(simp.coords)

def heading_variance(bearings):
    b = [normalize_bearing(x) for x in bearings if not pd.isna(x)]
    if len(b) == 0: return np.nan
    return float(np.var(b))

def trajectory_roughness(coords):
    # sum of absolute turn angles divided by length km
    if len(coords) < 3: return 0.0
    def bearing(p1, p2):
        lat1, lon1 = math.radians(p1[1]), math.radians(p1[0])
        lat2, lon2 = math.radians(p2[1]), math.radians(p2[0])
        dlon = lon2 - lon1
        y = math.sin(dlon) * math.cos(lat2)
        x = math.cos(lat1)*math.cos(lat2)*math.cos(dlon) + math.sin(lat1)*math.sin(lat2)
        brng = math.degrees(math.atan2(y, x))
        return (brng + 360) % 360
    bearings = [bearing(coords[i], coords[i+1]) for i in range(len(coords)-1)]
    turns = [abs(((bearings[i+1]-bearings[i]+180)%360)-180) for i in range(len(bearings)-1)]
    total_turn = sum(turns)
    length_km = line_length_km(coords)
    if length_km == 0: return 0.0
    return total_turn / length_km

def mask_id(raw_id):
    h = hashlib.sha1(str(raw_id).encode()).hexdigest()[:10]
    return f'id_{h}'

def k_suppress(df, count_col, k=K_ANON):
    return df[df[count_col] >= k].copy()

def h3_cell_polygon(lat, lng, res):
    cid = h3.geo_to_h3(lat, lng, res)
    boundary = h3.h3_to_geo_boundary(cid, geo_json=True)
    return cid, boundary

def frechet_distance(c1, c2):
    if not HAS_SIMILARITY:
        return None
    # similaritymeasures expects Nx2 arrays (x,y). We'll treat lon=x, lat=y.
    a = np.array([[p[0], p[1]] for p in c1])
    b = np.array([[p[0], p[1]] for p in c2])
    try:
        return similaritymeasures.frechet_dist(a, b)
    except Exception:
        return None

def pick_median_route(route_list):
    # route_list: list of coordinate sequences. If Frechet available, choose route minimizing avg dist. Else fallback heuristic length median.
    if len(route_list) == 0:
        return []
    if HAS_SIMILARITY and len(route_list) > 1:
        dists_matrix = []
        for i, r1 in enumerate(route_list):
            row = []
            for j, r2 in enumerate(route_list):
                if i == j:
                    row.append(0.0)
                else:
                    fd = frechet_distance(r1, r2)
                    fd = fd if fd is not None else 0.0
                    row.append(fd)
            dists_matrix.append(row)
        avg_dists = [np.mean(row) for row in dists_matrix]
        idx = int(np.argmin(avg_dists))
        return route_list[idx]
    # Fallback: pick route with median number of points (representative simplicity).
    lengths = [len(r) for r in route_list]
    med_len = np.median(lengths)
    idx = min(range(len(route_list)), key=lambda i: abs(lengths[i]-med_len))
    return route_list[idx]

print('Helper functions ready (with H3 compatibility).')

Helper functions ready (with H3 compatibility).


In [7]:
# -------------------------------------------------------------
# 4. Load CSV & Schema Checks
# -------------------------------------------------------------
expected_cols = ['randomized_id','lat','lng','alt','spd','azm']
df = pd.read_csv(DATA_PATH)
print(f'Loaded {len(df)} rows.')
missing_cols = [c for c in expected_cols if c not in df.columns]
if missing_cols:
    raise ValueError(f'Missing required columns: {missing_cols}')
df = df[expected_cols].copy()
# Dtypes
for c in ['lat','lng','alt','spd','azm']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

before_na = len(df)
df.dropna(subset=['randomized_id','lat','lng'], inplace=True)
print(f'Dropped {before_na - len(df)} rows with NA critical fields.')
before_bounds = len(df)
mask_bbox = (df['lat']>=BBOX['min_lat']) & (df['lat']<=BBOX['max_lat']) & (df['lng']>=BBOX['min_lng']) & (df['lng']<=BBOX['max_lng'])
df = df[mask_bbox].copy()
print(f'Out-of-bounds rows removed: {before_bounds - len(df)}')
# Duplicates
before_dup = len(df)
df.drop_duplicates(inplace=True)
print(f'Duplicates removed: {before_dup - len(df)}')
if len(df)==0:
    raise ValueError('No data after filtering.')
print(df.head())

Loaded 1262687 rows.
Dropped 0 rows with NA critical fields.
Out-of-bounds rows removed: 0
Duplicates removed: 72465
         randomized_id        lat        lng         alt        spd  \
0  7637058049336049989  51.095460  71.427530  350.531020   0.206810   
1  1259981924615926140  51.098200  71.412950  348.801610   0.000000   
2  1259981924615926140  51.098460  71.412120  349.273880   4.345010   
3  7180852955221959108  51.089779  71.428469  314.000000  14.326102   
4 -6683155579225977143  51.088782  71.417462  325.300018   0.000602   

          azm  
0   13.601680  
1  265.677000  
2  307.245300  
3  192.123672  
4    0.000000  


In [8]:
# -------------------------------------------------------------
# 5. Sequence Index & Basic Stats
# -------------------------------------------------------------
# Add sequence index per randomized_id preserving original order
df['row_order'] = np.arange(len(df))
df.sort_values(['randomized_id','row_order'], inplace=True)
df['seq_idx'] = df.groupby('randomized_id').cumcount()

# Normalize bearings
df['bearing'] = df['azm'].apply(normalize_bearing)
# Speed & altitude distributions
fig, axes = plt.subplots(1,3, figsize=(15,4))
df['spd'].dropna().hist(ax=axes[0], bins=40)
axes[0].set_title('Speed Distribution')
df['alt'].dropna().hist(ax=axes[1], bins=40, color='orange')
axes[1].set_title('Altitude Distribution')
df['bearing'].dropna().hist(ax=axes[2], bins=36, color='green')
axes[2].set_title('Bearing Distribution')
plt.tight_layout()
fig_path = os.path.join(OUTPUT_DIR,'fig_distributions.png')
plt.savefig(fig_path, dpi=150)
plt.close(fig)
print(f'Saved distributions plot to {fig_path}')
print(df[['spd','alt','bearing']].describe().T)

Saved distributions plot to outputs\fig_distributions.png
             count        mean         std    min         25%         50%  \
spd      1190222.0    7.216874    5.894857   -1.0    1.187060    6.781900   
alt      1190222.0  326.870034   18.609871 -191.4  317.400024  318.834709   
bearing  1190222.0  135.603353  109.994888    0.0   13.033230  106.285740   

                75%         max  
spd       12.409584   84.469543  
alt      347.700012  931.599280  
bearing  219.706889  359.999969  


In [9]:
# -------------------------------------------------------------
# 6. GeoDataFrame & Trajectory Construction (Raw + Simplified)
# -------------------------------------------------------------
gdf = gpd.GeoDataFrame(df.copy(), geometry=[Point(xy) for xy in zip(df['lng'], df['lat'])], crs='EPSG:4326')

# Build per-trip trajectories (LineString) in file order
trip_geoms = []
for tid, grp in gdf.groupby('randomized_id'):
    coords = list(zip(grp['lng'].values, grp['lat'].values))
    if len(coords) < 2: continue
    ls = LineString(coords)
    simp_coords = simplify_linestring(coords, tolerance=DP_TOLERANCE_DEG)
    ls_simp = LineString(simp_coords)
    # Additional 20m smoothing simplification
    ls_simp20_coords = simplify_linestring(coords, tolerance=DP_SMOOTH_TOL_DEG)
    ls_simp20 = LineString(ls_simp20_coords) if len(ls_simp20_coords) >= 2 else ls_simp
    trip_geoms.append({
        'randomized_id': tid,
        'geometry': ls,
        'geometry_simplified': ls_simp,
        'geometry_simplified_20m': ls_simp20,
        'num_points': len(coords),
        'num_points_simplified': len(simp_coords),
        'num_points_simplified_20m': len(ls_simp20_coords)
    })
trip_gdf = gpd.GeoDataFrame(trip_geoms, crs='EPSG:4326')
trip_gdf['length_km_est'] = trip_gdf['geometry'].apply(lambda g: line_length_km(list(g.coords)))
print(trip_gdf.head())
print(f'Trajectories built: {len(trip_gdf)} (with dual simplifications)')

         randomized_id                                           geometry  \
0 -9221304899272910788  LINESTRING (71.42236 51.08296, 71.42254 51.083...   
1 -9217374206810770265  LINESTRING (71.40246 51.08895, 71.40672 51.100...   
2 -9214548556609186054  LINESTRING (71.42192 51.07942, 71.42517 51.078...   
3 -9214033164510198912  LINESTRING (71.40579 51.09854, 71.40448 51.098...   
4 -9212938812549517684  LINESTRING (71.41194 51.0977, 71.41729 51.0969...   

                                 geometry_simplified  \
0  LINESTRING (71.4223554 51.0829583, 71.4194807 ...   
1  LINESTRING (71.4024592 51.0889499, 71.4067199 ...   
2  LINESTRING (71.4219152 51.0794236, 71.4269576 ...   
3  LINESTRING (71.4057889 51.0985433, 71.4044781 ...   
4  LINESTRING (71.4119383 51.0977033, 71.4172933 ...   

                             geometry_simplified_20m  num_points  \
0  LINESTRING (71.4223554 51.0829583, 71.4225399 ...         209   
1  LINESTRING (71.4024592 51.0889499, 71.4067199 ...          53

In [10]:
# -------------------------------------------------------------
# 7. H3 Aggregations (Counts & Unique Trips)
# -------------------------------------------------------------
# Using unified wrapper h3_cell_index for compatibility.

h3_records = []
for res in H3_RES_LIST:
    cids = [h3_cell_index(lat, lng, res) for lat, lng in zip(gdf['lat'].values, gdf['lng'].values)]
    gdf[f'h3_{res}'] = cids
    agg = gdf.groupby(f'h3_{res}').agg(
        point_count=('randomized_id','size'),
        unique_trips=('randomized_id','nunique')
    ).reset_index().rename(columns={f'h3_{res}':'h3_index'})
    agg['resolution'] = res
    h3_records.append(agg)

h3_all = pd.concat(h3_records, ignore_index=True)
h3_all['suppressed'] = h3_all['point_count'] < K_ANON
suppressed_pct = 100.0 * h3_all['suppressed'].mean()
h3_public = h3_all[~h3_all['suppressed']].copy()
# Precompute color scaling reference for later visualization if needed
h3_public['color_scale'] = (h3_public['point_count'] / h3_public['point_count'].max()).clip(0,1)

h3_all.to_csv(os.path.join(OUTPUT_DIR,'h3_aggregates_raw.csv'), index=False)
h3_public.to_csv(os.path.join(OUTPUT_DIR,'h3_aggregates.csv'), index=False)
print(f'H3 aggregated. Suppressed cells (%): {suppressed_pct:.2f}. Public cells saved -> outputs/h3_aggregates.csv')
h3_public.head()

H3 aggregated. Suppressed cells (%): 2.20. Public cells saved -> outputs/h3_aggregates.csv


,h3_index,point_count,unique_trips,resolution,suppressed,color_scale
0,872153806ffffff,498609,5141,7,False,1.000000
1,872153815ffffff,8537,959,7,False,0.017122
2,872153831ffffff,340259,4396,7,False,0.682416
3,872153833ffffff,342817,4058,7,False,0.687547
4,8821538061fffff,40462,1948,8,False,0.081150


In [11]:
# -------------------------------------------------------------
# 8. Start & End Cells + OD Flows (k-anon, filtered)
# -------------------------------------------------------------
# For each trip, identify start and end H3 cell at PRIMARY_HEAT_RES
res_col = f'h3_{PRIMARY_HEAT_RES}'
if res_col not in gdf.columns:
    raise ValueError('Primary resolution not computed.')
trip_bounds = gdf.sort_values(['randomized_id','seq_idx']).groupby('randomized_id').agg(
    start_cell=(res_col,'first'),
    end_cell=(res_col,'last'),
    start_lat=('lat','first'), start_lng=('lng','first'),
    end_lat=('lat','last'), end_lng=('lng','last')
).reset_index()
# OD pairs
od = trip_bounds.groupby(['start_cell','end_cell']).size().reset_index(name='trip_count')
# Drop self edges
od = od[od['start_cell'] != od['end_cell']].copy()
# Geodesic distance (fallback to haversine_km)

def _center_latlng(cell):
    try:
        return h3_cell_center(cell)
    except Exception:
        if hasattr(h3,'h3_to_geo'): return h3.h3_to_geo(cell)
        return (np.nan,np.nan)

start_centers = od['start_cell'].apply(_center_latlng)
end_centers = od['end_cell'].apply(_center_latlng)
od['start_lat'] = start_centers.apply(lambda x: x[0])
od['start_lng'] = start_centers.apply(lambda x: x[1])
od['end_lat'] = end_centers.apply(lambda x: x[0])
od['end_lng'] = end_centers.apply(lambda x: x[1])

def _dist_row(r):
    return haversine_km(r['start_lat'], r['start_lng'], r['end_lat'], r['end_lng'])
od['geodesic_km'] = od.apply(_dist_row, axis=1)
# Keep edges >=1 km
od = od[od['geodesic_km'] >= 1.0].copy()
# Apply k-anon on trip_count
od_public = k_suppress(od, 'trip_count', K_ANON)
od_public = od_public.sort_values('trip_count', ascending=False)
od_public.to_csv(os.path.join(OUTPUT_DIR,'od_public.csv'), index=False)
print('OD table saved (filtered & suppressed) -> outputs/od_public.csv')
trip_bounds.head()

OD table saved (filtered & suppressed) -> outputs/od_public.csv


,randomized_id,start_cell,end_cell,start_lat,start_lng,end_lat,end_lng
0,-9221304899272910788,8821538333fffff,8821538317fffff,51.082958,71.422355,51.094996,71.416271
1,-9217374206810770265,882153833bfffff,8821538331fffff,51.088950,71.402459,51.082103,71.400054
2,-9214548556609186054,8821538333fffff,8821538333fffff,51.079424,71.421915,51.079512,71.421314
3,-9214033164510198912,8821538311fffff,8821538317fffff,51.098543,71.405789,51.097647,71.412581
4,-9212938812549517684,8821538317fffff,8821538069fffff,51.097703,71.411938,51.094448,71.424033


In [12]:
# -------------------------------------------------------------
# 9. Stop Detection & Clustering (DBSCAN, revised thresholds)
# -------------------------------------------------------------
# Speed quantiles & unit inference
spd_series = df['spd'].dropna()
if spd_series.empty:
    raise ValueError('No speed data available for stop detection.')
quantiles = spd_series.quantile([0.5, 0.9, 0.95])
print('Speed quantiles (0.5,0.9,0.95):')
print(quantiles)

# Infer units: heuristic—if 95th percentile > 70 we assume km/h; else m/s
p95 = quantiles.loc[0.95]
units = 'km/h' if p95 > 70 else 'm/s'
print(f'Inferred speed units: {units}')

q10 = spd_series.quantile(0.10)
if units == 'm/s':
    STOP_SPEED_THRESHOLD = max(q10, 0.5)
else:  # km/h
    STOP_SPEED_THRESHOLD = max(q10, 2.0)
print(f'Using stop speed threshold: {STOP_SPEED_THRESHOLD} ({units})')

# Select stop-like points
if units == 'km/h' and p95 < 150:
    # Convert to m/s for clustering logic if needed (optional)
    # Not strictly necessary; DBSCAN uses coordinates only.
    pass

stop_points = gdf[(gdf['spd'] <= STOP_SPEED_THRESHOLD) & (~gdf['spd'].isna())].copy()
print(f'Stop-like points: {len(stop_points)}')

# Revised DBSCAN params per task: eps=40m, min_samples=20
from sklearn.cluster import DBSCAN
if len(stop_points) > 0:
    coords_rad = np.radians(stop_points[['lat','lng']].values)
    eps_rad = 40.0 / EARTH_RADIUS_M  # 40m
    db = DBSCAN(eps=eps_rad, min_samples=20, metric='haversine').fit(coords_rad)
    stop_points['cluster_id'] = db.labels_
    stop_clusters = stop_points[stop_points['cluster_id']>=0].groupby('cluster_id').agg(
        count=('cluster_id','size'),
        lat_mean=('lat','mean'),
        lng_mean=('lng','mean')
    ).reset_index().sort_values('count', ascending=False)
else:
    stop_clusters = pd.DataFrame(columns=['cluster_id','count','lat_mean','lng_mean'])

# Export top clusters
stop_clusters.to_csv(os.path.join(OUTPUT_DIR,'stop_clusters.csv'), index=False)
print('Stop clusters saved -> outputs/stop_clusters.csv')
stop_clusters.head()

Speed quantiles (0.5,0.9,0.95):
0.50     6.781900
0.90    15.439958
0.95    16.618267
Name: spd, dtype: float64
Inferred speed units: m/s
Using stop speed threshold: 0.5 (m/s)
Stop-like points: 241980
Stop clusters saved -> outputs/stop_clusters.csv


,cluster_id,count,lat_mean,lng_mean
0,0,230088,51.092078,71.417406
1,1,4085,51.082454,71.400974
9,9,1362,51.090628,71.399660
10,10,1214,51.088145,71.396739
2,2,905,51.089920,71.433184


In [13]:
# -------------------------------------------------------------
# 10. OD Graph from Start/End Clusters & Corridor Extraction (updated)
# -------------------------------------------------------------
# Derive start and end clusters by snapping trip start/end points to stop clusters (if available).
# If no stop clusters, fallback to H3 cells as clusters.
if len(stop_clusters) > 0:
    cl_coords = stop_clusters[['lat_mean','lng_mean']].values
    def assign_cluster(lat, lng):
        d = np.sqrt(((cl_coords - np.array([lat,lng]))**2).sum(axis=1))
        idx = np.argmin(d)
        return int(stop_clusters.iloc[idx]['cluster_id'])
    trip_bounds['start_cluster'] = trip_bounds.apply(lambda r: assign_cluster(r['start_lat'], r['start_lng']), axis=1)
    trip_bounds['end_cluster'] = trip_bounds.apply(lambda r: assign_cluster(r['end_lat'], r['end_lng']), axis=1)
    cluster_mode = 'stop_clusters'
else:
    trip_bounds['start_cluster'] = trip_bounds['start_cell']
    trip_bounds['end_cluster'] = trip_bounds['end_cell']
    cluster_mode = 'h3_cells'
print(f'Cluster mode for corridors: {cluster_mode}')

cluster_od = trip_bounds.groupby(['start_cluster','end_cluster']).size().reset_index(name='trip_count')
cluster_od = cluster_od.sort_values('trip_count', ascending=False)
# Top K for corridor modeling
top_cluster_od = cluster_od.head(TOP_K_OD)

# Build median route per top OD pair with 10-20 sampled trips (bounded by availability)
corridor_rows = []
for _, row in top_cluster_od.iterrows():
    sc, ec = row['start_cluster'], row['end_cluster']
    candidate_ids = trip_bounds[(trip_bounds['start_cluster']==sc) & (trip_bounds['end_cluster']==ec)]['randomized_id'].tolist()
    if not candidate_ids:
        continue
    random.shuffle(candidate_ids)
    sample_upper = min(len(candidate_ids), 20)
    sample_lower = min(sample_upper, 10)
    sample_n = random.randint(sample_lower, sample_upper)
    sample_ids = candidate_ids[:sample_n]
    route_coords = []
    for tid in sample_ids:
        tg = trip_gdf[trip_gdf['randomized_id']==tid]
        if tg.empty: continue
        # Prefer 20m simplified geometry
        geom = tg.iloc[0]['geometry_simplified_20m'] if 'geometry_simplified_20m' in tg.columns else tg.iloc[0]['geometry_simplified']
        coords = list(geom.coords)
        route_coords.append(coords)
    median_route = pick_median_route(route_coords)
    # Post-simplify median route again with 20m tolerance
    if median_route:
        median_route = simplify_linestring(median_route, tolerance=DP_SMOOTH_TOL_DEG)
    length_km = line_length_km(median_route) if median_route else 0
    heading_var = heading_variance(df[df['randomized_id'].isin(sample_ids)]['bearing'].values)
    corridor_rows.append({
        'start_cluster': sc, 'end_cluster': ec, 'trip_count': row['trip_count'],
        'median_length_km_est': length_km, 'heading_variance': heading_var,
        'median_route_coords': json.dumps(median_route)
    })

corridors_df = pd.DataFrame(corridor_rows).sort_values('trip_count', ascending=False)
corridors_df.to_csv(os.path.join(OUTPUT_DIR,'od_top.csv'), index=False)
print('Top OD corridors saved -> outputs/od_top.csv (updated sampling & smoothing)')
corridors_df.head()

Cluster mode for corridors: stop_clusters
Top OD corridors saved -> outputs/od_top.csv (updated sampling & smoothing)


,start_cluster,end_cluster,trip_count,median_length_km_est,heading_variance,median_route_coords
0,16,16,771,0.646176,10043.231991,"[[71.42552, 51.09525], [71.42551, 51.09565], [..."
1,35,35,328,0.373936,11682.989365,"[[71.4189819, 51.1010976], [71.4175334, 51.101..."
2,7,7,327,46.483733,9141.817720,"[[71.4060345, 51.098903], [71.4100269, 51.0940..."
3,24,24,146,0.000000,12530.271437,"[[71.40894, 51.09177], [71.40894, 51.09177]]"
4,23,23,115,0.203830,12544.675996,"[[71.4209155, 51.0771191], [71.4221887, 51.077..."


In [14]:
# -------------------------------------------------------------
# 11. Sample Trajectories (Masked, smoothed bearings)
# -------------------------------------------------------------
all_trip_ids = trip_gdf['randomized_id'].unique().tolist()
random.shuffle(all_trip_ids)
sample_ids = all_trip_ids[:SAMPLE_N_TRIPS] if len(all_trip_ids) >= SAMPLE_N_TRIPS else all_trip_ids
sample_rows = []

# Helper to compute rolling median bearing
def rolling_median_bearing(bearings, window=5):
    s = pd.Series(bearings)
    return list(s.rolling(window, center=True, min_periods=1).median().fillna(method='bfill').fillna(method='ffill'))

for tid in sample_ids:
    rec = trip_gdf[trip_gdf['randomized_id']==tid]
    if rec.empty: continue
    geom = rec.iloc[0]['geometry_simplified_20m'] if 'geometry_simplified_20m' in rec.columns else rec.iloc[0]['geometry_simplified']
    coords = list(geom.coords)
    length_km = line_length_km(coords)
    # Derive bearings pairwise
    def _bearing(p1, p2):
        lat1, lon1 = math.radians(p1[1]), math.radians(p1[0])
        lat2, lon2 = math.radians(p2[1]), math.radians(p2[0])
        dlon = lon2 - lon1
        y = math.sin(dlon) * math.cos(lat2)
        x = math.cos(lat1)*math.cos(lat2)*math.cos(dlon) + math.sin(lat1)*math.sin(lat2)
        brng = math.degrees(math.atan2(y, x))
        return (brng + 360) % 360
    raw_bearings = [_bearing(coords[i], coords[i+1]) for i in range(len(coords)-1)] if len(coords)>1 else []
    sm_bearings = rolling_median_bearing(raw_bearings, window=5) if raw_bearings else []
    # Roughness uses smoothed bearings
    turns = [abs(((sm_bearings[i+1]-sm_bearings[i]+180)%360)-180) for i in range(len(sm_bearings)-1)] if len(sm_bearings)>1 else []
    total_turn = sum(turns)
    roughness = (total_turn / length_km) if length_km>0 else 0.0
    heading_var = float(np.var(sm_bearings)) if sm_bearings else np.nan
    spd_vals = df[df['randomized_id']==tid]['spd'].dropna()
    sample_rows.append({
        'masked_id': mask_id(tid),
        'num_points': rec.iloc[0]['num_points'],
        'length_km_est': length_km,
        'heading_variance_smoothed': heading_var,
        'roughness_smoothed': roughness,
        'speed_min': float(spd_vals.min()) if len(spd_vals)>0 else np.nan,
        'speed_max': float(spd_vals.max()) if len(spd_vals)>0 else np.nan,
        'speed_mean': float(spd_vals.mean()) if len(spd_vals)>0 else np.nan
    })

sample_trips_df = pd.DataFrame(sample_rows)
sample_trips_df.to_csv(os.path.join(OUTPUT_DIR,'sample_trips.csv'), index=False)
print('Sample trips summary saved -> outputs/sample_trips.csv (smoothed bearings)')
sample_trips_df

Sample trips summary saved -> outputs/sample_trips.csv (smoothed bearings)


,masked_id,num_points,length_km_est,heading_variance_smoothed,roughness_smoothed,speed_min,speed_max,speed_mean
0,id_63d0010818,194,33.182718,31829.168722,10.854564,-1.000000e+00,17.864580,5.999484
1,id_cb7dbea102,95,29.785898,24853.759650,12.093351,2.294280e+00,17.017410,8.984177
2,id_99f5b8e328,52,16.535889,30942.640381,43.546950,9.626100e-01,18.189130,9.087746
3,id_7087812908,11,0.679121,24799.727871,1060.194873,1.548310e+00,4.370430,2.289645
4,id_fb1bb8b2f2,87,34.005697,26598.435498,21.179153,4.215661e-19,17.343159,9.304517


In [15]:
# -------------------------------------------------------------
# 12. Anomaly Detection (Updated: Top 1% Roughness & GPS Jumps)
# -------------------------------------------------------------
# This cell supersedes any earlier anomaly logic. It computes:
#  - Per-trip geometric roughness (using 20m simplified geometry if available)
#  - GPS jump counts (>300m instantaneous displacement)
#  - Flags top 1% roughness (global) and any jump occurrences
#  - Exports anomaly_metrics.csv with columns: randomized_id, roughness, gps_jumps, flags
#  - Derives per-cell (primary H3 resolution) GPS jump counts for later visualization

from math import radians, sin, cos, sqrt, atan2

if 'trip_gdf' not in globals():
    raise RuntimeError('trip_gdf not present; run trajectory construction cell before anomaly detection.')

# Helper: haversine distance in km
def _hv_km(a, b):
    lat1, lon1 = a
    lat2, lon2 = b
    R = 6371.0088
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    sa = sin(dlat/2)**2 + cos(radians(lat1))*cos(radians(lat2))*sin(dlon/2)**2
    c = 2*atan2(sqrt(sa), sqrt(1-sa))
    return R*c

# Ensure primary H3 index on base points for jump aggregation later
primary_col = f'h3_{PRIMARY_HEAT_RES}'
if primary_col not in gdf.columns:
    gdf[primary_col] = [h3_cell_index(lat,lng,PRIMARY_HEAT_RES) for lat,lng in zip(gdf['lat'], gdf['lng'])]

# Sort base dataframe for jump detection
if 'seq_idx' in gdf.columns:
    gdf_sorted = gdf.sort_values(['randomized_id','seq_idx']).copy()
else:
    gdf_sorted = gdf.sort_values(['randomized_id']).copy()

# Compute per-point previous coordinates
gdf_sorted['prev_lat'] = gdf_sorted.groupby('randomized_id')['lat'].shift(1)
gdf_sorted['prev_lng'] = gdf_sorted.groupby('randomized_id')['lng'].shift(1)
mask_valid_prev = gdf_sorted['prev_lat'].notna()

# Distance per step (km)
step_dist_km = []
for lat, lng, plat, plng in zip(gdf_sorted['lat'], gdf_sorted['lng'], gdf_sorted['prev_lat'], gdf_sorted['prev_lng']):
    if pd.isna(plat):
        step_dist_km.append(0.0)
    else:
        step_dist_km.append(_hv_km((plat, plng),(lat,lng)))

gdf_sorted['step_dist_km'] = step_dist_km
# Flag GPS jumps > 0.3 km (300 m)
JUMP_THRESHOLD_KM = 0.3
gdf_sorted['jump_flag'] = gdf_sorted['step_dist_km'] > JUMP_THRESHOLD_KM

# Aggregate jumps per trip
trip_jump_counts = gdf_sorted.groupby('randomized_id')['jump_flag'].sum().rename('gps_jumps').reset_index()

# Build anomaly metrics base using trip_gdf (one row per trip)
rows = []
for _, row in trip_gdf.iterrows():
    tid = row['randomized_id']
    # choose geometry_simplified_20m if present else geometry_simplified else geometry
    geom = None
    if 'geometry_simplified_20m' in row and row['geometry_simplified_20m'] is not None:
        geom = row['geometry_simplified_20m']
    elif 'geometry_simplified' in row and row['geometry_simplified'] is not None:
        geom = row['geometry_simplified']
    else:
        geom = row['geometry']
    coords = list(geom.coords) if geom is not None else []
    rough = trajectory_roughness(coords) if coords else 0.0
    rows.append({'randomized_id': tid, 'roughness': rough})

anom_new = pd.DataFrame(rows)
anom_new = anom_new.merge(trip_jump_counts, on='randomized_id', how='left')
anom_new['gps_jumps'] = anom_new['gps_jumps'].fillna(0).astype(int)

# Determine top 1% roughness threshold (guard for few trips)
if len(anom_new) >= 50:
    rough_thresh = anom_new['roughness'].quantile(0.99)
elif len(anom_new) > 5:
    rough_thresh = anom_new['roughness'].quantile(0.95)
else:
    rough_thresh = anom_new['roughness'].max()

flags_all = []
for _, r in anom_new.iterrows():
    f = []
    if r['roughness'] >= rough_thresh and rough_thresh > 0:
        f.append('ROUGH')
    if r['gps_jumps'] > 0:
        f.append('JUMPS')
    flags_all.append(';'.join(f))

anom_new['flags'] = flags_all

# Export with required schema
anom_export = anom_new[['randomized_id','roughness','gps_jumps','flags']].copy()
anom_export.to_csv(os.path.join(OUTPUT_DIR,'anomaly_metrics.csv'), index=False)
print(f"Anomaly metrics (updated) saved -> outputs/anomaly_metrics.csv. Roughness top threshold={rough_thresh:.4f} Trips flagged={ (anom_new['flags']!='').sum() }")

# Per-cell GPS jump aggregation (assign jump to cell of the END point of the jump step)
# We could also assign to start; choice documented here.
jump_points = gdf_sorted[gdf_sorted['jump_flag']].copy()
if not jump_points.empty:
    jump_counts_cell = jump_points.groupby(primary_col).size().reset_index(name='gps_jumps')
else:
    jump_counts_cell = pd.DataFrame(columns=[primary_col,'gps_jumps'])

# Store global for heatmap merge
globals()['h3_jump_counts'] = jump_counts_cell

# Optionally merge into existing h3_public (non-destructive; missing cells get 0)
if 'h3_public' in globals() and not h3_public.empty:
    if primary_col in gdf.columns:
        # Only modify resolution == PRIMARY_HEAT_RES copy, not original object structure integrity
        try:
            h3_public.loc[h3_public['resolution']==PRIMARY_HEAT_RES,'gps_jumps'] = h3_public[h3_public['resolution']==PRIMARY_HEAT_RES]['h3_index'].map(
                dict(zip(jump_counts_cell[primary_col], jump_counts_cell['gps_jumps']))
            ).fillna(0).astype(int)
        except Exception as me:
            print(f'[Warn] Could not annotate h3_public with gps_jumps: {me}')



Anomaly metrics (updated) saved -> outputs/anomaly_metrics.csv. Roughness top threshold=0.0168 Trips flagged=5666


In [16]:
# -------------------------------------------------------------
# 12. Anomaly Detection (Geometry-only)
# -------------------------------------------------------------
anomaly_rows = []
for idx, row in trip_gdf.iterrows():
    coords = list(row['geometry_simplified'].coords)
    roughness = trajectory_roughness(coords)
    hvar = heading_variance(df[df['randomized_id']==row['randomized_id']]['bearing'].values)
    anomaly_rows.append({'randomized_id': row['randomized_id'], 'roughness': roughness, 'heading_var': hvar, 'length_km': row['length_km_est']})
anom_df = pd.DataFrame(anomaly_rows)
# z-scores
for col in ['roughness','heading_var']:
    mu = anom_df[col].mean()
    sd = anom_df[col].std() if anom_df[col].std() else 1.0
    anom_df[f'{col}_z'] = (anom_df[col]-mu)/sd
anom_df['rough_flag'] = anom_df['roughness_z'].abs() > ROUGHNESS_Z_THRESHOLD
anom_df['heading_flag'] = anom_df['heading_var_z'].abs() > HEADING_VAR_Z_THRESHOLD
anom_df['any_anomaly'] = anom_df[['rough_flag','heading_flag']].any(axis=1)
num_anomalies = anom_df['any_anomaly'].sum()
anom_df.to_csv(os.path.join(OUTPUT_DIR,'anomaly_metrics.csv'), index=False)
print(f'Anomaly metrics saved -> outputs/anomaly_metrics.csv. Flagged trajectories: {num_anomalies}')
anom_df.head()

Anomaly metrics saved -> outputs/anomaly_metrics.csv. Flagged trajectories: 45


,randomized_id,roughness,heading_var,length_km,roughness_z,heading_var_z,rough_flag,heading_flag,any_anomaly
0,-9221304899272910788,0.005062,14175.411036,130.020940,-0.744721,1.090835,False,False,False
1,-9217374206810770265,0.003620,4.392627,34.348924,-1.047998,-1.575722,False,False,False
2,-9214548556609186054,0.011514,1682.629543,6.712419,0.612453,-1.259929,False,False,False
3,-9214033164510198912,0.016809,10678.613458,51.912318,1.726223,0.432843,False,False,False
4,-9212938812549517684,0.008703,12694.647713,207.408486,0.021061,0.812200,False,False,False


In [17]:
# -------------------------------------------------------------
# 13. Visualization: H3 Heatmap (pydeck) + Static PNG (Hardened)
#     Extended to dual maps: point_count & unique_trips + GPS jumps tooltip
# -------------------------------------------------------------
import json as _json

try:
    # Defensive (re)definition of H3 wrapper helpers
    try:
        h3  # noqa: F821
    except NameError:
        import h3  # type: ignore

    if 'h3_cell_center' not in globals():
        def h3_cell_center(cell):
            if hasattr(h3, 'h3_to_geo'):
                return h3.h3_to_geo(cell)
            if hasattr(h3, 'cell_to_latlng'):
                return h3.cell_to_latlng(cell)
            raise AttributeError('No H3 center function available')

    if 'h3_cell_boundary' not in globals():
        def h3_cell_boundary(cell):
            if hasattr(h3, 'h3_to_geo_boundary'):
                return h3.h3_to_geo_boundary(cell, geo_json=True)
            if hasattr(h3, 'cell_to_boundary'):
                return h3.cell_to_boundary(cell, geo_json=True)
            raise AttributeError('No H3 boundary function available')

    # Use integrity utilities if present
    if 'ensure_h3_public' in globals():
        h3_public_local = ensure_h3_public()
    else:
        if 'h3_public' not in globals() or 'h3_all' not in globals():
            print('[Reconstruct:Fallback] rebuilding minimal h3_public (integrity utilities absent).')
            if 'gdf' not in globals():
                raise RuntimeError('gdf not available; run earlier preprocessing cells.')
            recs = []
            for res in H3_RES_LIST if 'H3_RES_LIST' in globals() else [PRIMARY_HEAT_RES]:
                col = f'h3_{res}'
                if col not in gdf.columns:
                    gdf[col] = [h3_cell_index(lat,lng,res) for lat,lng in zip(gdf['lat'], gdf['lng'])]
                agg = gdf.groupby(col).agg(point_count=('randomized_id','size'), unique_trips=('randomized_id','nunique')).reset_index().rename(columns={col:'h3_index'})
                agg['resolution'] = res
                recs.append(agg)
            all_df = pd.concat(recs, ignore_index=True)
            all_df['suppressed'] = all_df['point_count'] < K_ANON
            h3_public_local = all_df[~all_df['suppressed']].copy()
            if h3_public_local['point_count'].max() == 0:
                h3_public_local['color_scale'] = 0.0
            else:
                h3_public_local['color_scale'] = (h3_public_local['point_count']/h3_public_local['point_count'].max()).clip(0,1)
            globals()['h3_all'] = all_df
            globals()['h3_public'] = h3_public_local
        else:
            h3_public_local = h3_public

    # Attach gps_jumps if we have per-cell counts
    primary_col = f'h3_{PRIMARY_HEAT_RES}'
    if 'h3_jump_counts' in globals():
        jc_map = dict(zip(h3_jump_counts[primary_col], h3_jump_counts['gps_jumps']))
        # Only apply to primary resolution rows
        if 'resolution' in h3_public_local.columns:
            mask_primary = h3_public_local['resolution']==PRIMARY_HEAT_RES
            h3_public_local.loc[mask_primary,'gps_jumps'] = h3_public_local.loc[mask_primary,'h3_index'].map(jc_map).fillna(0).astype(int)
    if 'gps_jumps' not in h3_public_local.columns:
        h3_public_local['gps_jumps'] = 0

    # Ensure color_scale
    if 'color_scale' not in h3_public_local.columns:
        if h3_public_local['point_count'].max() == 0:
            h3_public_local['color_scale'] = 0.0
        else:
            h3_public_local['color_scale'] = (h3_public_local['point_count']/h3_public_local['point_count'].max()).clip(0,1)

    heat_res = PRIMARY_HEAT_RES
    res_subset = h3_public_local[h3_public_local['resolution']==heat_res].copy()

    # Add centers + fill colors for point density map
    if HAS_PYDECK and len(res_subset)>0:
        try:
            res_subset['lat'] = res_subset['h3_index'].apply(lambda x: h3_cell_center(x)[0])
            res_subset['lng'] = res_subset['h3_index'].apply(lambda x: h3_cell_center(x)[1])
        except Exception as ce:
            print(f'[Warn] Center computation failed: {ce}; defaulting to zeros.')
            res_subset['lat'] = 0.0
            res_subset['lng'] = 0.0
        # point_count color
        if 'color_scale' not in res_subset.columns:
            if res_subset['point_count'].max() == 0:
                res_subset['color_scale'] = 0.0
            else:
                res_subset['color_scale'] = (res_subset['point_count'] / res_subset['point_count'].max()).clip(0,1)
        res_subset['fill_r'] = 255
        res_subset['fill_g'] = (res_subset['color_scale']*255).astype(int).clip(0,255)
        res_subset['fill_b'] = 100
        res_subset['fill_a'] = 180
        res_subset['fill_color'] = res_subset.apply(lambda r: [int(r['fill_r']), int(r['fill_g']), int(r['fill_b']), int(r['fill_a'])], axis=1)
        tooltip_text = 'Cell: {h3_index}\\nPoints: {point_count}\\nTrips: {unique_trips}\\nJumps: {gps_jumps}'
        try:
            heat_layer = pdk.Layer(
                'H3HexagonLayer',
                data=res_subset,
                get_hexagon='h3_index',
                get_fill_color='fill_color',
                pickable=True,
                auto_highlight=True,
                extruded=True,
                elevation_scale=2,
                get_elevation='point_count'
            )
            view_state = pdk.ViewState(latitude=51.169, longitude=71.449, zoom=10.5, pitch=40)
            deck = pdk.Deck(layers=[heat_layer], initial_view_state=view_state, tooltip={'text': tooltip_text})
            map_path = os.path.join(OUTPUT_DIR,'map_h3_points.html')
            deck.to_html(map_path, css_background_color='white')
            print(f'pydeck point_count heatmap saved -> {map_path}')
        except Exception as pe:
            print(f'[Error] pydeck point_count rendering failed: {pe}')

        # Unique trips heatmap variant
        try:
            res_subset['color_scale_trips'] = (res_subset['unique_trips']/res_subset['unique_trips'].max()).replace([np.inf, -np.inf], 0).fillna(0)
            res_subset['fill_color_trips'] = res_subset.apply(lambda r: [int(100 + 155*r['color_scale_trips']), int(50 + 205*(1-r['color_scale_trips'])), 150, 180], axis=1)
            tooltip_text_trips = 'Cell: {h3_index}\\nTrips: {unique_trips}\\nPoints: {point_count}\\nJumps: {gps_jumps}'
            trips_layer = pdk.Layer(
                'H3HexagonLayer',
                data=res_subset,
                get_hexagon='h3_index',
                get_fill_color='fill_color_trips',
                pickable=True,
                auto_highlight=True,
                extruded=True,
                elevation_scale=2,
                get_elevation='unique_trips'
            )
            deck_trips = pdk.Deck(layers=[trips_layer], initial_view_state=view_state, tooltip={'text': tooltip_text_trips})
            map_path_trips = os.path.join(OUTPUT_DIR,'map_h3_trips.html')
            deck_trips.to_html(map_path_trips, css_background_color='white')
            print(f'pydeck unique_trips heatmap saved -> {map_path_trips}')
        except Exception as pe2:
            print(f'[Error] pydeck unique_trips rendering failed: {pe2}')
    else:
        print('pydeck not available or no data; skipping interactive heatmaps.')

    # Static polygon plot for point_count
    try:
        poly_records = []
        for _, r in res_subset.iterrows():
            try:
                boundary = h3_cell_boundary(r['h3_index'])
                if not boundary: continue
                poly = LineString(boundary + [boundary[0]]).buffer(0)
                poly_records.append({'geometry': poly, 'point_count': r['point_count'], 'unique_trips': r['unique_trips']})
            except Exception as be:
                print(f'[Warn] boundary failed for cell {r.get("h3_index")}: {be}')
        if poly_records:
            poly_gdf = gpd.GeoDataFrame(poly_records, crs='EPSG:4326')
            fig, ax = plt.subplots(figsize=(8,8))
            poly_gdf.plot(column='point_count', ax=ax, legend=True, cmap='viridis', linewidth=0.1, edgecolor='grey')
            ax.set_title('H3 Point Density (Suppression Applied)')
            plt.axis('off')
            png_path = os.path.join(OUTPUT_DIR,'fig_h3_points.png')
            plt.savefig(png_path, dpi=150, bbox_inches='tight')
            plt.close(fig)
            print(f'Static point_count heatmap PNG saved -> {png_path}')
            # Unique trips static
            fig2, ax2 = plt.subplots(figsize=(8,8))
            poly_gdf.plot(column='unique_trips', ax=ax2, legend=True, cmap='plasma', linewidth=0.1, edgecolor='grey')
            ax2.set_title('H3 Unique Trips Density (Suppression Applied)')
            plt.axis('off')
            png_path2 = os.path.join(OUTPUT_DIR,'fig_h3_trips.png')
            plt.savefig(png_path2, dpi=150, bbox_inches='tight')
            plt.close(fig2)
            print(f'Static unique_trips heatmap PNG saved -> {png_path2}')
        else:
            print('No polygons to render for static heatmaps.')
    except Exception as sp:
        print(f'[Error] Static polygon rendering failed: {sp}')

except Exception as e:
    print(f'[Fatal] Visualization cell encountered unrecoverable error: {e}')

pydeck point_count heatmap saved -> outputs\map_h3_points.html
pydeck unique_trips heatmap saved -> outputs\map_h3_trips.html
[Warn] boundary failed for cell 8821538061fffff: cell_to_boundary() got an unexpected keyword argument 'geo_json'
[Warn] boundary failed for cell 8821538065fffff: cell_to_boundary() got an unexpected keyword argument 'geo_json'
[Warn] boundary failed for cell 8821538069fffff: cell_to_boundary() got an unexpected keyword argument 'geo_json'
[Warn] boundary failed for cell 882153806bfffff: cell_to_boundary() got an unexpected keyword argument 'geo_json'
[Warn] boundary failed for cell 882153806dfffff: cell_to_boundary() got an unexpected keyword argument 'geo_json'
[Warn] boundary failed for cell 8821538159fffff: cell_to_boundary() got an unexpected keyword argument 'geo_json'
[Warn] boundary failed for cell 8821538311fffff: cell_to_boundary() got an unexpected keyword argument 'geo_json'
[Warn] boundary failed for cell 8821538313fffff: cell_to_boundary() got an u

In [18]:
# -------------------------------------------------------------
# 12b. (Integrity) Resilience Utilities & Self-Check
# -------------------------------------------------------------
import types

def _df_exists(name):
    return name in globals() and isinstance(globals()[name], (pd.DataFrame, gpd.GeoDataFrame))

def ensure_trip_bounds():
    if _df_exists('trip_bounds'): return trip_bounds
    if not _df_exists('gdf') or 'seq_idx' not in gdf.columns:
        raise RuntimeError('gdf/seq_idx not ready; run earlier cells.')
    res_col = f'h3_{PRIMARY_HEAT_RES}'
    if res_col not in gdf.columns:
        # minimal compute
        gdf[res_col] = [h3_cell_index(lat,lng,PRIMARY_HEAT_RES) for lat,lng in zip(gdf['lat'], gdf['lng'])]
    tb = gdf.sort_values(['randomized_id','seq_idx']).groupby('randomized_id').agg(
        start_cell=(res_col,'first'),
        end_cell=(res_col,'last'),
        start_lat=('lat','first'), start_lng=('lng','first'),
        end_lat=('lat','last'), end_lng=('lng','last')
    ).reset_index()
    globals()['trip_bounds'] = tb
    return tb

def ensure_h3_public():
    if _df_exists('h3_public') and 'color_scale' in h3_public.columns:
        return h3_public
    if not _df_exists('gdf'):
        raise RuntimeError('gdf missing; cannot rebuild H3 aggregates.')
    recs = []
    for res in H3_RES_LIST:
        col = f'h3_{res}'
        if col not in gdf.columns:
            gdf[col] = [h3_cell_index(lat,lng,res) for lat,lng in zip(gdf['lat'], gdf['lng'])]
        agg = gdf.groupby(col).agg(point_count=('randomized_id','size'), unique_trips=('randomized_id','nunique')).reset_index().rename(columns={col:'h3_index'})
        agg['resolution'] = res
        recs.append(agg)
    all_df = pd.concat(recs, ignore_index=True)
    all_df['suppressed'] = all_df['point_count'] < K_ANON
    pub = all_df[~all_df['suppressed']].copy()
    if pub['point_count'].max() == 0:
        pub['color_scale'] = 0.0
    else:
        pub['color_scale'] = (pub['point_count']/pub['point_count'].max()).clip(0,1)
    globals()['h3_all'] = all_df
    globals()['h3_public'] = pub
    return pub

def ensure_corridors():
    if _df_exists('corridors_df'): return corridors_df
    tb = ensure_trip_bounds()
    if not _df_exists('trip_gdf'):
        raise RuntimeError('trip_gdf missing; run trajectory cell.')
    # minimal corridor build using existing logic (top K only)
    cluster_mode = 'h3_cells'
    if 'start_cluster' not in tb.columns:
        tb['start_cluster'] = tb['start_cell']
        tb['end_cluster'] = tb['end_cell']
    cluster_od = tb.groupby(['start_cluster','end_cluster']).size().reset_index(name='trip_count').sort_values('trip_count', ascending=False)
    top_cluster_od = cluster_od.head(TOP_K_OD)
    rows = []
    for _, row in top_cluster_od.iterrows():
        sc, ec = row['start_cluster'], row['end_cluster']
        ids = tb[(tb['start_cluster']==sc) & (tb['end_cluster']==ec)]['randomized_id'].tolist()
        random.shuffle(ids)
        sample_ids = ids[:ROUTE_SAMPLE_PER_OD]
        routes = []
        for tid in sample_ids:
            tg = trip_gdf[trip_gdf['randomized_id']==tid]
            if tg.empty: continue
            coords = list(tg.iloc[0]['geometry_simplified'].coords) if 'geometry_simplified' in tg.columns else list(tg.iloc[0]['geometry'].coords)
            routes.append(coords)
        median_route = pick_median_route(routes)
        length_km = line_length_km(median_route) if median_route else 0
        heading_var = heading_variance(df[df['randomized_id'].isin(sample_ids)]['bearing'].values)
        rows.append({
            'start_cluster': sc, 'end_cluster': ec, 'trip_count': row['trip_count'],
            'median_length_km_est': length_km, 'heading_variance': heading_var,
            'median_route_coords': json.dumps(median_route)
        })
    cd = pd.DataFrame(rows).sort_values('trip_count', ascending=False)
    globals()['corridors_df'] = cd
    return cd

def ensure_sample_trips():
    if _df_exists('sample_trips_df'): return sample_trips_df
    if not _df_exists('trip_gdf'):
        raise RuntimeError('trip_gdf missing; run trajectory cell.')
    tids = trip_gdf['randomized_id'].unique().tolist()
    random.shuffle(tids)
    sel = tids[:SAMPLE_N_TRIPS] if len(tids) >= SAMPLE_N_TRIPS else tids
    rows = []
    for tid in sel:
        rec = trip_gdf[trip_gdf['randomized_id']==tid]
        if rec.empty: continue
        coords = list(rec.iloc[0]['geometry'].coords)
        length_km = line_length_km(coords)
        roughness = trajectory_roughness(coords)
        heading_var = heading_variance(df[df['randomized_id']==tid]['bearing'].values)
        spd_vals = df[df['randomized_id']==tid]['spd'].dropna()
        rows.append({
            'masked_id': mask_id(tid),
            'num_points': rec.iloc[0].get('num_points', len(coords)),
            'length_km_est': length_km,
            'heading_variance': heading_var,
            'roughness': roughness,
            'speed_min': float(spd_vals.min()) if len(spd_vals)>0 else np.nan,
            'speed_max': float(spd_vals.max()) if len(spd_vals)>0 else np.nan,
            'speed_mean': float(spd_vals.mean()) if len(spd_vals)>0 else np.nan
        })
    sdf = pd.DataFrame(rows)
    globals()['sample_trips_df'] = sdf
    return sdf

# Quick self-check (will not raise unless critical)
try:
    _ = ensure_h3_public()
    print('[Integrity] h3_public OK')
except Exception as e:
    print(f'[Integrity] h3_public rebuild failed: {e}')


[Integrity] h3_public OK


In [19]:
# -------------------------------------------------------------
# 14. Visualization: Start/End Clusters, Corridors, Sample Trips (Hardened)
# -------------------------------------------------------------
try:
    center_lat, center_lng = 51.169, 71.449

    # Defensive redefinition of mask_id if earlier cell skipped
    if 'mask_id' not in globals():
        import hashlib
        def mask_id(raw_id):
            h = hashlib.sha1(str(raw_id).encode()).hexdigest()[:10]
            return f'id_{h}'

    # Rehydrate dependencies using integrity utilities if present
    if 'ensure_trip_bounds' in globals():
        trip_bounds_local = ensure_trip_bounds()
    else:
        trip_bounds_local = trip_bounds if 'trip_bounds' in globals() else None

    if trip_bounds_local is None:
        raise RuntimeError('trip_bounds unavailable and integrity utility missing.')

    if 'ensure_corridors' in globals():
        corridors_local = ensure_corridors()
    else:
        corridors_local = corridors_df if 'corridors_df' in globals() else pd.DataFrame(columns=['start_cluster','end_cluster','trip_count','median_route_coords','median_length_km_est'])

    if 'ensure_sample_trips' in globals():
        sample_trips_local = ensure_sample_trips()
    else:
        sample_trips_local = sample_trips_df if 'sample_trips_df' in globals() else pd.DataFrame()

    # Guarantee cluster columns
    if 'start_cluster' not in trip_bounds_local.columns:
        trip_bounds_local['start_cluster'] = trip_bounds_local.get('start_cell', trip_bounds_local.columns[0])
    if 'end_cluster' not in trip_bounds_local.columns:
        trip_bounds_local['end_cluster'] = trip_bounds_local.get('end_cell', trip_bounds_local.columns[1])

    # Start/End cluster markers
    if HAS_FOLIUM:
        try:
            m_clusters = folium.Map(location=[center_lat, center_lng], zoom_start=11, tiles='cartodbpositron')
            sc_counts = trip_bounds_local.groupby('start_cluster').size().reset_index(name='count').sort_values('count', ascending=False)
            for _, r in sc_counts.iterrows():
                subset = trip_bounds_local[trip_bounds_local['start_cluster']==r['start_cluster']].iloc[0]
                folium.CircleMarker(location=[subset['start_lat'], subset['start_lng']], radius=4+math.log1p(r['count']),
                                    popup=f'Start Cluster {r["start_cluster"]} Count {r["count"]}', color='blue', fill=True).add_to(m_clusters)
            ec_counts = trip_bounds_local.groupby('end_cluster').size().reset_index(name='count').sort_values('count', ascending=False)
            for _, r in ec_counts.iterrows():
                subset = trip_bounds_local[trip_bounds_local['end_cluster']==r['end_cluster']].iloc[0]
                folium.CircleMarker(location=[subset['end_lat'], subset['end_lng']], radius=4+math.log1p(r['count']),
                                    popup=f'End Cluster {r["end_cluster"]} Count {r["count"]}', color='red', fill=True).add_to(m_clusters)
            path_clusters = os.path.join(OUTPUT_DIR,'map_start_end_clusters.html')
            m_clusters.save(path_clusters)
            print(f'Start/End clusters map saved -> {path_clusters}')
        except Exception as mc:
            print(f'[Warn] Cluster map failed: {mc}')
    else:
        print('Folium not available; skipping start/end clusters map.')

    # Corridors map
    if HAS_FOLIUM and not corridors_local.empty:
        try:
            m_corridors = folium.Map(location=[center_lat, center_lng], zoom_start=11, tiles='cartodbpositron')
            max_count = corridors_local['trip_count'].max() if 'trip_count' in corridors_local.columns and len(corridors_local)>0 else 1
            for _, r in corridors_local.iterrows():
                if 'median_route_coords' not in r or pd.isna(r['median_route_coords']):
                    continue
                coords = _json.loads(r['median_route_coords']) if isinstance(r['median_route_coords'], str) else []
                if len(coords) < 2: continue
                weight = 1 + 9*((r['trip_count']/max_count) if max_count else 1)
                folium.PolyLine([(c[1], c[0]) for c in coords], color='#3186cc', weight=weight,
                                popup=f'OD {r.get("start_cluster")}->{r.get("end_cluster")} Trips {r.get("trip_count")} LengthKm {r.get("median_length_km_est",0):.2f}').add_to(m_corridors)
            path_corr = os.path.join(OUTPUT_DIR,'map_corridors.html')
            m_corridors.save(path_corr)
            print(f'Corridors map saved -> {path_corr}')
        except Exception as co:
            print(f'[Warn] Corridors map failed: {co}')
    else:
        print('Skipping corridors map (folium unavailable or empty corridors).')

    # Sample trips map
    if HAS_FOLIUM and not sample_trips_local.empty:
        try:
            m_samples = folium.Map(location=[center_lat, center_lng], zoom_start=11, tiles='cartodbpositron')
            color_palette = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']
            ids_iter = []
            if 'randomized_id' in trip_gdf.columns:
                # reconstruct sample_ids from sample_trips_local masked ids if necessary
                ids_iter = trip_gdf['randomized_id'].unique().tolist()[:len(sample_trips_local)]
            for i, tid in enumerate(ids_iter):
                tg = trip_gdf[trip_gdf['randomized_id']==tid]
                if tg.empty: continue
                if 'geometry_simplified' in tg.columns:
                    coords = list(tg.iloc[0]['geometry_simplified'].coords)
                else:
                    coords = list(tg.iloc[0]['geometry'].coords)
                folium.PolyLine([(c[1], c[0]) for c in coords], color=color_palette[i%len(color_palette)], weight=3,
                                popup=mask_id(tid)).add_to(m_samples)
            path_samples = os.path.join(OUTPUT_DIR,'map_sample_trips.html')
            m_samples.save(path_samples)
            print(f'Sample trips map saved -> {path_samples}')
        except Exception as sm:
            print(f'[Warn] Sample trips map failed: {sm}')
    else:
        print('Skipping sample trips map (folium unavailable or no samples).')

except Exception as e:
    print(f'[Fatal] Visualization (clusters/corridors/sample trips) error: {e}')

Start/End clusters map saved -> outputs\map_start_end_clusters.html
Corridors map saved -> outputs\map_corridors.html
Sample trips map saved -> outputs\map_sample_trips.html


In [20]:
# -------------------------------------------------------------
# 15. Metrics & Usable Cases Summary
# -------------------------------------------------------------
top_cells = h3_public[h3_public['resolution']==PRIMARY_HEAT_RES].sort_values('unique_trips', ascending=False).head(10)
print('Top H3 cells (by unique trips):')
display(top_cells[['h3_index','point_count','unique_trips']])

# Top start & end clusters
if 'start_cluster' in trip_bounds.columns:
    start_cluster_counts = trip_bounds.groupby('start_cluster').size().reset_index(name='trip_count').sort_values('trip_count', ascending=False).head(10)
    end_cluster_counts = trip_bounds.groupby('end_cluster').size().reset_index(name='trip_count').sort_values('trip_count', ascending=False).head(10)
    print('Top Start Clusters:')
    display(start_cluster_counts)
    print('Top End Clusters:')
    display(end_cluster_counts)

print('Top OD Corridors:')
display(corridors_df[['start_cluster','end_cluster','trip_count','median_length_km_est']].head(TOP_K_OD))

# % suppressed at primary res
res_all = h3_all[h3_all['resolution']==PRIMARY_HEAT_RES]
sup_pct_primary = 100.0*res_all['suppressed'].mean()
print(f'% of cells suppressed (primary res {PRIMARY_HEAT_RES}): {sup_pct_primary:.2f}%')
anom_count = pd.read_csv(os.path.join(OUTPUT_DIR,'anomaly_metrics.csv'))['any_anomaly'].sum()
print(f'Flagged rough/heading anomaly trajectories: {anom_count}')

# Markdown style summary
usable_cases_md = f'''
### What this dataset enables (No-Timestamp Context)
- Demand heat & spatial intensity via H3 cell counts (ride/trip density).
- Candidate pickup/drop hotspots (stop clusters) for repositioning & infrastructure.
- Origin-Destination corridors for planning optimized shuttle/dispatch routes.
- Route geometry variability & anomaly cues (zig-zag / roughness) highlight potential map errors, GPS noise zones, or safety review areas.
- Simplified representative ("median") routes for top OD pairs assisting navigation template generation.

### Key Limitations
- No temporal metrics: cannot compute travel time, dwell time, speed validation over distance, or peak-hour patterns.
- Speed column accepted as-is (cannot verify with segment distance/time).
- No dwell threshold confidence: stop detection purely speed-based.
'''
print(usable_cases_md)
with open(os.path.join(OUTPUT_DIR,'summary_usable_cases.md'),'w', encoding='utf-8') as f:
    f.write(usable_cases_md)
print('Summary markdown saved -> outputs/summary_usable_cases.md')

Top H3 cells (by unique trips):


,h3_index,point_count,unique_trips
6,8821538069fffff,232469,4169
13,8821538317fffff,193511,3680
10,8821538311fffff,55589,3228
8,882153806dfffff,165024,2860
15,8821538333fffff,121707,2617
18,882153833bfffff,176135,2606
14,8821538331fffff,58032,2204
11,8821538313fffff,47295,1961
4,8821538061fffff,40462,1948
12,8821538315fffff,31969,1864


Top Start Clusters:


,start_cluster,trip_count
16,16,1349
7,7,872
35,35,651
19,19,394
23,23,378
27,27,359
1,1,319
29,29,296
24,24,255
21,21,183


Top End Clusters:


,end_cluster,trip_count
15,16,1597
7,7,682
34,35,529
23,24,369
21,22,353
28,29,304
18,19,278
22,23,268
26,27,267
1,1,263


Top OD Corridors:


,start_cluster,end_cluster,trip_count,median_length_km_est
0,16,16,771,0.646176
1,35,35,328,0.373936
2,7,7,327,46.483733
3,24,24,146,0.000000
4,23,23,115,0.203830


% of cells suppressed (primary res 8): 0.00%
Flagged rough/heading anomaly trajectories: 45

### What this dataset enables (No-Timestamp Context)
- Demand heat & spatial intensity via H3 cell counts (ride/trip density).
- Candidate pickup/drop hotspots (stop clusters) for repositioning & infrastructure.
- Origin-Destination corridors for planning optimized shuttle/dispatch routes.
- Route geometry variability & anomaly cues (zig-zag / roughness) highlight potential map errors, GPS noise zones, or safety review areas.
- Simplified representative ("median") routes for top OD pairs assisting navigation template generation.

### Key Limitations
- No temporal metrics: cannot compute travel time, dwell time, speed validation over distance, or peak-hour patterns.
- Speed column accepted as-is (cannot verify with segment distance/time).
- No dwell threshold confidence: stop detection purely speed-based.

Summary markdown saved -> outputs/summary_usable_cases.md


In [21]:
# -------------------------------------------------------------
# 16. Outputs Inventory
# -------------------------------------------------------------
for fn in sorted(os.listdir(OUTPUT_DIR)):
    print(fn)
print('Notebook complete. All artifacts written to outputs/.')

anomaly_metrics.csv
config_snapshot.json
fig_distributions.png
h3_aggregates.csv
h3_aggregates_raw.csv
map_corridors.html
map_h3_heat.html
map_h3_points.html
map_h3_trips.html
map_sample_trips.html
map_start_end_clusters.html
od_public.csv
od_top.csv
sample_trips.csv
stop_clusters.csv
summary_usable_cases.md
Notebook complete. All artifacts written to outputs/.
